# Processing Nanopore reads

Set variables

In [ ]:
SEQ_SUMMARY_DIR="./sequencing_summary"
FASTQ_PASS_DIR = "./fastq_pass"
WORK_DIR = "./work"

## Quality control

In [ ]:
!NanoStat --summary ./sequencing_summary/*_sequencing_summary.txt --readtype 1D

## Demultiplexing using qcat

In [ ]:
!cat ./fastq_pass/*.fastq | qcat -b ./work --trim

## Make a list of barcodes

I extract the detected barcodes from the `work` directory to use later.

In [ ]:
import glob
all_files = glob.glob("work/*.fastq")
fastq_files = glob.glob("work/barcode*.fastq")
barcodes = [fq.split("/")[1].split(".")[0] for fq in fastq_files]
barcodes

## Summarise each output file

In [ ]:
for f in all_files:
    !echo {f};NanoStat -t 16 --fastq {f}; echo "\n"

## Identify references using Mash

The following uses a special curly brace syntax to loop through the barcodes.

In [ ]:
for bc in barcodes:
    !echo {bc};mash screen -p 16 -w refs/denv_chikv.msh work/{bc}.fastq > work/{bc}.screen

The following code sorts the Mash results, finds the top reference, and extracts it from the FASTA file.

In [ ]:
for bc in barcodes:
    !samtools faidx refs/denv_chikv.fas `sort -gr work/{bc}.screen | cut -f 5 | head -1` > work/{bc}_ref.fa

## Map using graphmap

In [ ]:
for bc in barcodes:
    !graphmap align -t 16 -r work/{bc}_ref.fa -d work/{bc}.fastq -o work/{bc}.graphmap.sam

## Convert, sort and index the BAM files

In [ ]:
for bc in barcodes:
    !samtools view -bS work/{bc}.graphmap.sam | samtools sort - -o work/{bc}.graphmap.bam

In [ ]:
for bc in barcodes:
    !samtools index work/{bc}.graphmap.bam

## Get coverage from BAM file

In [ ]:
for bc in barcodes:
    !mosdepth -t 16 work/{bc} work/{bc}.graphmap.bam

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=[16,16])
for i in range(len(barcodes)):
    bc = barcodes[i]
    plt.subplot(len(barcodes), 1, i+1)
    df = pd.read_table("work/"+bc+".per-base.bed.gz",compression='gzip',sep='\t',header=None)
    plt.yscale('log')
    plt.title(bc)
    plt.step(df[1],df[3])

## Extract consensus from each BAM

In [ ]:
for bc in barcodes:
    !echo {bc}
    !kindel consensus work/{bc}.graphmap.bam > work/{bc}_consensus.fa

## Concatenate draft assemblies

In [ ]:
from Bio import SeqIO
records = []
blacklist = set(["barcode24"])
for bc in barcodes:
    if bc in blacklist:
        continue
    record=SeqIO.read("work/"+bc+"_consensus.fa",format="fasta")
    record.id=bc
    record.name=bc
    record.description=bc
    records.append(record)
SeqIO.write(records,"work/consensus.fa",format="fasta")

## Concatenate sequences with references

In [ ]:
!cat refs/chikv.fas work/consensus.fa > work/consensus_withrefs.fa

## Align sequences

In [ ]:
!fftnsi --thread 16 work/consensus_withrefs.fa > work/consensus_withrefs.fa.fftnsi

## Make a quick tree

In [ ]:
!iqtree -s work/consensus_withrefs.fa.fftnsi -m GTR+G4 -nt 4 -pre work/consensus_withrefs -fast